如下 pipeline 设计无关于具体的游戏类型，也无关于 UI 交互的实现，仅是策划案生成和代码生成的实现。

由于 Agent 的响应速度慢，我们希望通过人工设计好调用规则流程，省去用 Agent 判断、切换和使用工具的步骤，而直接让模型专注于每个节点的工作。

In [1]:
from openai import OpenAI
import streamlit as st

OPENAI_API_KEY = st.secrets["openai_api_key"]

由于多个 pipeline 对应要有多个 history memory，因此需要一个 MemoryManager 来实例化多个 node 的 memory，且该 memory 是可以在超出最大 token 限制时自动总结历史记录的。

In [2]:
from openai import OpenAI
import tiktoken

class Memory:
    def __init__(self, openai_api_key, 
        max_tokens=128000, 
        compressed_tokens=4000,
        temperature=0.3
    ):
        self.memory = []
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.compressed_tokens = compressed_tokens
        self.client = OpenAI(api_key=openai_api_key)
        self.encoding = tiktoken.get_encoding("cl100k_base")
    
    def count_tokens(self, text: str) -> int:
        return len(self.encoding.encode(text))
    
    def count_memory_tokens(self, skip_first=False) -> int:
        start_idx = 1 if skip_first and len(self.memory) > 0 else 0
        return sum(msg["tokens"] for msg in self.memory[start_idx:])
    
    def summarize_history(self) -> str:
        assert len(self.memory) > 1 
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.memory[1:]])

        summarize_history_prompt = (
            "如下是用户与游戏策划师的历史对话。"
            "请用简洁且准确的语言总结该对话历史，保留用户每次提问和策划师每次回答与修改的关键信息。"
        )

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": summarize_history_prompt},
                {"role": "user", "content": history_text}
            ],
            temperature=self.temperature,
            max_tokens=self.compressed_tokens
        )

        return response.choices[0].message.content
    
    def add(self, role: str, content: str) -> None:
        self.memory.append({
            "role": role,  
            "content": content,
            "tokens": self.count_tokens(content)
        })
        
        if self.count_memory_tokens(skip_first=False) > self.max_tokens:
            first_message = self.memory[0]
            summary = self.summarize_history()
            self.memory = [first_message]
            self.memory.append({
                "role": "system",
                "content": f"历史对话总结如下：\n{summary}",
                "tokens": self.count_tokens(summary)
            })

    def pop(self) -> None:
        if self.memory:
            self.memory.pop()

我们如下称全部调用过程为 pipeline，称各节点为 nodes。

首先是 plan pipeline 的实现。交互时，用户可能存在两种询问，一种需要 nodes 直接生成按照格式定义的策划案内容，另一种询问可能是对当前策划过程的疑惑，因此需要 nodes 回答自然语言，以解答用户的疑惑或给出建议。

这里 plan pipeline 通过三个 node 实现，一个 node 负责识别意图并选择不同的回答 response format，一个 node 回答输出，一个 node 生成更新策划。共享同一个 memory，但切换 node 的 IO 不会写入到 memory 中。

首先实现两个 prompt。

In [3]:
plan_base_prompt = """
你是一个专业的辅助用户实现 Unity 游戏开发的游戏策划师。你应当礼貌地拒绝任何与游戏开发无关的交流。
你的用户通常是一些初学者，他们希望使用 Unity 进行游戏开发。他们对脑中期望游戏的描述通常是比较抽象的想法，而不是具体的游戏细节。
你需要根据你的专业知识，发挥充分的想象力，将用户的抽象想法细化成具体的“游戏玩法”和“场景搭建”。其内容是游戏场景搭建的详细步骤，以及对游戏策划案的详细描述。确保游戏策划和场景搭建的完整性和一致性，不要遗漏。

如果用户提出了游戏策划的生成想法、修改意见，说明此时应当修改游戏策划；如果用户在与你交流沟通、询问建议等，说明此时应当先回答用户。
如下是三个策划案生成的具体例子，仅作为参考。请注意，你生成的“所需代码”的结构大致决定了用户需要写入的代码结构，因此请务必确保所需代码的结构是合理、独立且完整的。

#### 示例 1
- 用户的想法：我想要一个玩家控制小鸟飞行并躲避柱子的游戏。
- **场景搭建**
```json
[
    "放置小鸟图片，并在小鸟上加载小鸟控制代码",
    "放置柱子图片，并在柱子上加载柱子控制代码",
    "放置背景图片，并在背景上加载背景控制代码",
    "创建游戏控制空物体，并在游戏控制空物体上加载游戏控制代码",
    "创建UI，并加载UI控制代码"
]
```
- **游戏策划**
```json
{
    "游戏玩法": "玩家点击屏幕触发小鸟飞起，否则小鸟下落。场景中会有连续不断的上下两根柱子向小鸟移动，玩家需要控制小鸟通过两根柱子之间的空隙，否则游戏结束。",
    "所需素材": {
        "Bird.png": "小鸟的图片",
        "Pipe.png": "柱子的图片",
        "Background.png": "背景的图片"
    },
    "所需代码": {
        "Bird.cs": "用于控制小鸟移动的代码",
        "Column.cs": "用于柱子基本实现的代码",
        "ColumnPool.cs": "生成和重用柱子的代码",
        "GameControl.cs": "用于管理整个游戏逻辑的代码",
        "RepeatingBackground.cs": "用于实现无限滚动背景的代码",
        "ScrollingObject.cs": "控制场景中物体向左移动以及碰撞检测的代码"
    }
}
```

#### 示例 2
- 用户的想法：你觉得一个玩家控制钩爪，抓取物品的游戏怎么样？物品可以是金矿，石头，钻石。
- **场景搭建**
```json
[
    "创建空物体，放置游戏控制代码",
    "搭建UI，放置UI控制代码",
    "放置玩家图片与钩爪图片，放置玩家控制代码",
    "创建金矿预制体、石头预制体、钻石预制体",
    "放置背景图片"
]
```
- **游戏策划**
```json
{
    "游戏玩法": "玩家点击屏幕触发左右钟式摇摆的钩爪，抛出钩爪向前飞出直到触碰到物品，将物品抓回并按照物品类型结算效果，游戏倒计时结束则游戏结束，积分数量决定是否进入下一关。物品分为金矿，个体大小不一，获得正常积分且抓回速度正常；石头，个体大，获得少量积分且抓回速度缓慢；钻石，个体小，获得大量积分且抓回速度快。",
    "所需素材": {
        "Hook.png": "钩爪的图片",
        "Rope.png": "钩爪绳子的图片",
        "Background.png": "背景的图片",
        "Player.png": "玩家的图片",
        "Gold.png": "金矿的图片",
        "Stone.png": "石头的图片",
        "Diamond.png": "钻石的图片"
    },
    "所需代码": {
        "GameControl.cs": "游戏控制代码",
        "PlayerControl.cs": "玩家控制代码",
        "ItemControl.cs": "物品控制代码",
        "UIControl.cs": "UI控制代码"
        ......
    }
}
```

#### 示例 3
- 用户的想法：我想做一个俄罗斯方块游戏。
- **场景搭建**
```json
[
    "创建一个网格 Grid，并设置大小",
    "创建一个棋盘空物体并放置 Board 代码与 Piece 代码，放置 Grid 组件，创建 Tilemap 子物体",
    "创建一个阴影空物体并放置 Ghost 代码，放置 Grid 组件，创建 Tilemap 子物体",
    "利用图片素材创建 Tile，为 Borad 代码添加 Tetrominoes 元素，构建方块"
]
```
- **游戏策划**
```json
{
    "游戏玩法": "俄罗斯方块（Tetris）的游戏玩法基本如下：游戏目标：玩家的目标是在一个矩形的游戏区域内，通过旋转和移动不同形状的方块，使这些方块在底部形成完整的横行。当一行被填满时，该行会消失，玩家获得分数。游戏继续进行，方块下落的速度会逐渐加快。游戏结束的条件是方块堆积到游戏区域的顶部，此时没有空间再放置新的方块。方块形状：游戏中共有七种不同形状的方块，每种方块由四个小方块组成，它们分别是：I, O, T, S, Z, J, L。方块操作：下落：方块会从游戏区域的顶部开始向下移动，玩家无法停止或加速这个下落过程。旋转：玩家可以按特定按钮使方块顺时针旋转90度。左右移动：玩家可以使用左右方向键移动方块，使其在水平方向上移动。快速下落：玩家可以按特定按钮（通常是向下方向键或空格键）使方块快速下落到当前可到达的最低位置。消除行：当一行被完全填满时，该行会被消除，玩家获得分数，并且上面的所有行都会下移一格。同时消除多行会获得更高的分数。得分：每消除一行，玩家获得一定的分数。消除多行可以获得额外的分数奖励。随着游戏进行，方块下落速度加快，玩家获得的分数也会更高。游戏结束：如果新的方块无法进入游戏区域，游戏结束。玩家可以看到自己的最终得分，并且可以选择重新开始游戏。俄罗斯方块是一个简单但极具挑战性的游戏，它考验玩家的空间想象力、反应速度和策略规划能力。通过不断练习，玩家可以提高自己的游戏技巧和得分。",
    "所需素材": {
        "Border.png": "棋盘边框的图片",
        "RedBlock.png": "红色小方块的图片",
        "BlueBlock.png": "蓝色小方块的图片",
        "GreenBlock.png": "绿色小方块的图片",
        "YellowBlock.png": "黄色小方块的图片",
        "PurpleBlock.png": "紫色小方块的图片",
        "CyanBlock.png": "青色小方块的图片",
        "OrangeBlock.png": "橙色小方块的图片"
    },
    "所需代码": {
        "Board.cs": "棋盘代码",
        "Piece.cs": "方块代码",
        "Ghost.cs": "阴影代码",
        "Tetrominoes.cs": "方块元素代码",
        "Data.cs": "静态数据类代码"
    }
}
```
"""

plan_switch_prompt = """
你是一个专业的辅助用户实现 Unity 游戏开发的游戏策划师。
请根据用户的输入，判断此时你应当直接生成或修改游戏策划，还是先回答用户。
如果用户提出了游戏策划的生成想法、修改意见，说明此时应当修改游戏策划；如果用户在与你交流沟通、询问建议等，说明此时应当先回答用户。

如果可以直接生成游戏策划，请直接输出数字 0；如果需要先回答用户，请直接输出数字 1。
请输出："""

实现两个输出格式。

In [4]:
plan_switch_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "plan_switch_schema",
        "schema": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "integer",
                    "description": "数字 0 或 1"
                }
            },
            "required": ["mode"]
        }
    }
}

plan_writing_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "plan_writing_schema",
        "schema": {
            "type": "object",
            "properties": {
                "游戏策划": {
                    "description": "游戏的详细策划，包括玩法、所需素材和代码",
                    "type": "object",
                    "properties": {
                        "游戏玩法": {"description": "游戏的玩法", "type": "string"},
                        "所需素材": {"description": "游戏所需的素材", "type": "object"},
                        "所需代码": {"description": "游戏所需的代码", "type": "object"}
                    }
                },
                "场景搭建": {
                    "description": "游戏的场景搭建",
                    "type": "array",
                    "items": {"description": "场景搭建步骤", "type": "string"}
                }
            }
        }
    }
}

实现三个 node 以及对应的输出装饰函数。

In [5]:
from abc import ABC, abstractmethod
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from openai import OpenAI, RateLimitError, APIConnectionError, AuthenticationError

class BaseNode(ABC):
    def __init__(self, openai_api_key, 
        model: str, 
        temperature: float, 
        memory: Memory,
        max_retries: int = 3,
        retry_delay: float = 1.0
    ):
        self.client = OpenAI(api_key=openai_api_key)
        self.model = model
        self.temperature = temperature
        self.memory = memory
        self.max_retries = max_retries
        self.retry_delay = retry_delay

    @abstractmethod
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=1, max=10),
        reraise=True
    )
    def step(self, query: str):
        pass

In [6]:
from functools import wraps
import json

def handle_completion(save_path=None, stream=False):
    def decorator(func):
        @wraps(func)
        def wrapper(self, query: str, *args, **kwargs):
            self.memory.add(role="user", content=query)
            completion = func(self, query, *args, **kwargs)
            
            if stream:
                response = ""
                for chunk in completion:
                    if chunk.choices[0].delta.content is not None:
                        content = chunk.choices[0].delta.content
                        print(content, end="")
                        response += content
            else:
                response = completion.choices[0].message.content

                if save_path and save_path.endswith('.json'):
                    response_dict = json.loads(response)
                    response_dict = json.loads(json.dumps(response_dict).replace('\\n', ''))

                    with open(save_path, 'w', encoding='utf-8') as f:
                        json.dump(response_dict, f, ensure_ascii=False, indent=4)
                    print("已更新", save_path, "中的策划案。")
                
                if save_path and save_path.endswith('.cs'):
                    # response_dict = json.loads(response)
                    # TODO

                    with open(save_path, 'w', encoding='utf-8') as f:
                        f.write(response)
                    print("已更新", save_path, "中的代码。")
                
            self.memory.add(role="assistant", content=response)
            return response
        return wrapper
    return decorator

In [7]:
class PlanSwitchNode(BaseNode):
    def __init__(self, openai_api_key, 
        memory: Memory,
        model: str = "gpt-4o-2024-08-06", 
        temperature: float = 0.0, 
        k: int = 3
    ):
        super().__init__(openai_api_key, model, temperature, memory)
        self.k = k

    def step(self, query: str):
        memory_ = self.memory.memory[:self.k]
        memory_.append({"role": "user", "content": query})
        memory_.append({"role": "system", "content": plan_switch_prompt})

        completion = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=memory_,
            response_format=plan_switch_format
        )

        mode = json.loads(completion.choices[0].message.content)["mode"]
        assert mode in [0, 1]
        return mode

class PlanWritingNode(BaseNode):
    def __init__(self, openai_api_key, 
        memory: Memory,
        model: str = "gpt-4o-2024-08-06", 
        temperature: float = 0.1, 
    ):
        super().__init__(openai_api_key, model, temperature, memory)

    @handle_completion(save_path="plan.json")
    def step(self, query: str):
        return self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=self.memory.memory,
            response_format=plan_writing_format
        )

class PlanAnswerNode(BaseNode):
    def __init__(self, openai_api_key, 
        memory: Memory,
        model: str = "gpt-4o-2024-08-06", 
        temperature: float = 0.2, 
    ):
        super().__init__(openai_api_key, model, temperature, memory)

    @handle_completion(stream=True)
    def step(self, query: str):
        return self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            stream=True,
            messages=self.memory.memory
        )

实现整个 plan pipeline。

In [8]:
class PlanPipeline:
    def __init__(self, openai_api_key):
        self.plan_memory = Memory(openai_api_key=openai_api_key, max_tokens=120000)
        self.plan_memory.add(role="system", content=plan_base_prompt)

        self.plan_switch_node = PlanSwitchNode(openai_api_key, memory=self.plan_memory)
        self.plan_writing_node = PlanWritingNode(openai_api_key, memory=self.plan_memory)
        self.plan_answer_node = PlanAnswerNode(openai_api_key, memory=self.plan_memory)

    def step(self, query: str):
        mode = self.plan_switch_node.step(query)
        if mode == 0:
            return self.plan_writing_node.step(query)
        else:
            return self.plan_answer_node.step(query)

在这里测试如上 plan pipeline 的效果。可以看到就算经过多轮的交互，plan pipeline 也能正确响应。

In [ ]:
plan_pipeline = PlanPipeline(openai_api_key=OPENAI_API_KEY)
_ = plan_pipeline.step("你好呀？")

In [ ]:
_ = plan_pipeline.step("我想要一个玩家控制一只大脚踩死虫子的游戏，虫子有快慢两种。")

In [ ]:
_ = plan_pipeline.step("踩死慢的虫子得1分，踩死快的虫子得2分。")

In [ ]:
_ = plan_pipeline.step("素材要再加上虫子被踩死之后爆浆的图片。")

In [ ]:
_ = plan_pipeline.step("你觉得当前的策划怎么样？")

In [ ]:
_ = plan_pipeline.step("连续踩死十只虫子奖励翻倍，连续踩死二十只虫子则获得一个变大脚的buff，持续十秒。")

In [ ]:
_ = plan_pipeline.step("还有没有什么可能的改进？")

In [ ]:
_ = plan_pipeline.step("那就加上一个跳跃机制，小虫子会跳跃，然后大虫子不会。")

In [2]:
"哪个文件实现了虫子生成", "把随机生成的逻辑改成70%概率生成慢虫子，30%生成快虫子"

('哪个文件实现了虫子生成', '把随机生成的逻辑改成70%概率生成慢虫子，30%生成快虫子')

In [ ]:
plan_pipeline.plan_memory.count_memory_tokens()
plan_pipeline.plan_memory.memory

粗略计算，开始时的 prompt token 在 2500 左右，之后每次如果用户询问或者闲聊，则 response token 每次增加 10 + 20 = 30 左右（不计算 switch 的 token，大概在 100 左右），如果用户提出新的需求，则 response token 增加 400 到 1000 左右。则多轮交互的次数上限为大约 1110 次，是非常充足的。

接下来，我们实现 Code 部分的 pipeline。这部分 pipeline 我们分初始阶段和后续阶段。
初始阶段，我们根据策划案生成代码库；后续阶段，我们根据用户的反馈修改代码，或者使用一些反思机制修改代码，使得它更加完善和鲁棒，从而真的实现游戏玩法。

我们考虑到初始阶段也有两种可能，一种是代码库还不存在，用户希望我们生成代码库；另一种是代码库已经存在，用户可能希望依据新的策划案更新代码库。

对于第一种情况，通常来说，假设 code pipeline 在初始阶段的代码就完全正确，那么按理说是不需要后续阶段的。因此我们肯定希望 code pipeline 在初始阶段就尽可能生成完全正确的代码库。按理说，我们应该默认用户不会再在之后的交互中加入策划需求了，更多可能是一些细节改动、反馈 bug 或者优化建议。但是单次的 code generation 是有可能不完美的，因此我们希望能有 node 用于人类用户或者其他 node 进行反思和修改。

单次 code generation 的 token 上限是 128k - 3k 的 prompt token，大约 125k 的 response token。如果最多十个代码文件的话，每个代码文件的 token 上限大约是 12.5k。StackCube(complex) 的总代码数是 8795 tokens，因此只是理论上，我们生成的游戏复杂度（难度）最多是 StackCube 的十倍。

当然也不排除用户在这一阶段产生了新的策划想法。这种情况我们可以这样处理，使用一个 node 专门判断当前用户的输入是不是一个新的策划设计需求，如果是则同时调用 plan node 修改策划案和 code node 修改代码库，如果不是，则剩下两种可能，一种是用户在反馈 bug 或者优化建议，另一种是用户在闲聊。对于后者，我们则希望 code node 能够忽略它。

也就是说，第二种情况的输入有两种，一种是新的策划案，另一种是用户的输入。我们需要分别处理它们。用户的输入还需要判断是 QA 还是代码局部修改。code pipeline 总共有七种（总结 memory 写入专门有一种）node，其中切换的小 node 有两个，摘要的小 node 有一个，一次性生成全部 codebase 的大 node 一个，ranking node 一个，单文件精修 node 一个。具体的 memory 读写机制和 codebase 读写我们之后再讨论。

除此以外我们还有两个功能考虑，一个是 revert 功能（同时使用 memory.pop 和 codebase 退回），一个是自我反思。自我反思功能是可选的。

In [ ]:
import os
import json

def load_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

plan_ready = load_json("plan.json")
plan_ready = plan_ready["游戏策划"]
plan_ready

In [9]:
code_base_prompt = f"""
你是一个辅助用户实现 Unity 游戏开发的 Unity C# 代码开发专家。
你的用户通常是一些初学者，他们希望使用 Unity Hub 进行游戏开发，但是对 Unity 的使用并不熟悉。

你需要充分理解用户对游戏策划和场景搭建的策划；其中，“游戏玩法”和“所需代码”是你需要重点关注的内容，你需要依据此像一个 C# 专家一样写代码；你被鼓励尽可能多生成代码；你应当尽可能完整的实现每个函数功能，不允许留有一些待实现的函数或代码块；你生成的.cs文件中每个自创的变量或类必须都有对应的实现，而不允许使用未创建的类实例化，也不允许编造不存在的变量，也不允许在某一.cs文件中声明某一类或函数，而后在另一.cs文件里调用它；你在写代码的时候如果要命名某个 class，那么该 class 的命名应当与当前文件名保持一致；

你不被允许引用不常用的第三方库，或者在代码中使用没有定义的类或函数或变量等；你可以使用 Unity 自带的库，例如 Unity，UnityEngine 和 UnityEditor 等，但需要按照 Unity Documentation Scripting API 的规范使用。

如下是一个例子。

假设已有如下的游戏策划：
- **游戏策划**
```json
{{
    "游戏玩法": "玩家点击屏幕触发小鸟飞起，否则小鸟下落。场景中会有连续不断的上下两根柱子向小鸟移动，玩家需要控制小鸟通过两根柱子之间的空隙，否则游戏结束。",
    "所需素材": {{
        "Bird.png": "小鸟的图片",
        "Pipe.png": "柱子的图片",
        "Background.png": "背景的图片"
    }},
    "所需代码": {{
        "Bird.cs": "用于控制小鸟移动的代码",
        "Column.cs": "用于柱子基本实现的代码",
        "ColumnPool.cs": "生成和重用柱子的代码",
        "GameControl.cs": "用于管理整个游戏逻辑的代码",
        "RepeatingBackground.cs": "用于实现无限滚动背景的代码",
        "ScrollingObject.cs": "控制场景中物体向左移动以及碰撞检测的代码"
    }}
}}
```
则你生成的代码库类似如下：
- Bird.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class Bird : MonoBehaviour
{{
    public float upForce = 200f;
    public int player_num;

    private bool isDead = false;
    private bool isDead2 = false;
    private Rigidbody2D rb2d;
    private Animator anim;
    
    void Start()
    {{
        rb2d = GetComponent<Rigidbody2D> ();
        anim = GetComponent<Animator> ();
    }}
    
    void Update()
    {{
        if (isDead == false)
        {{
            if (Input.GetMouseButtonDown(0) && player_num == 0) //left click
            {{
                rb2d.velocity = Vector2.zero;
                rb2d.AddForce(new Vector2 (0, upForce));
                anim.SetTrigger("Flap");
            }}
        }}
        if (isDead2 == false)
        {{
            if (Input.GetKeyDown(KeyCode.Space) && player_num == 1)
            {{
                rb2d.velocity = Vector2.zero;
                rb2d.AddForce(new Vector2(0, upForce));
                anim.SetTrigger("Flap");
            }}
        }}
    }}

    void OnCollisionEnter2D()
    {{
        if (isDead == false && player_num == 0)
        {{
            rb2d.velocity = Vector2.zero;
            isDead = true;
            anim.SetTrigger("Die");
            GameControl.instance.BirdDied();
        }}
    }}
}}
```
- Column.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class Column : MonoBehaviour {{
    private void OnTriggerEnter2D(Collider2D other)
    {{
        if (other.GetComponent<Bird> () != null)
        {{
            if (other.CompareTag("B1"))
            {{
                GameControl.instance.BirdScored();
            }}
        }}
    }}
}}
```
- ColumnPool.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class ColumnPool : MonoBehaviour {{
    public int columnPoolSize = 5;
    public GameObject columnPrefab;
    public float spawnRate = 4f;
    public float columnMin = -1f;
    public float columnMax = 3.5f;

    private GameObject[] columns;
    private Vector2 objectPoolPosition = new Vector2(-15f, -25f);
    private float timeSinceLastSpawned;
    private float spawnXPosition = 10f;
    private int currentColumn = 0;

	// Use this for initialization
	void Start () 
    {{
        columns = new GameObject[columnPoolSize];
        for (int i = 0; i < columnPoolSize; i++)
        {{
            columns[i] = (GameObject)Instantiate(columnPrefab, objectPoolPosition, Quaternion.identity);
        }}
	}}
	
	// Update is called once per frame
	void Update () 
    {{
        timeSinceLastSpawned += Time.deltaTime;
        if (timeSinceLastSpawned >= spawnRate)
        {{
             timeSinceLastSpawned = 0;
             float spawnYPosition = Random.Range(columnMin, columnMax);
             columns[currentColumn].transform.position = new Vector2(spawnXPosition, spawnYPosition);
             currentColumn++;
             if (currentColumn >= columnPoolSize)
             {{
                currentColumn = 0;
             }}
        }}
	}}
}}
```
- GameControl.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;
using UnityEngine.UI;

public class GameControl : MonoBehaviour {{

    public static GameControl instance;
    public GameObject gameOverText;
    public Text scoreText;
    public bool gameOver = false;
    public float scrollSpeed = -1.5f;
    private int score = 0;

	void Awake () 
    {{
        if (instance == null)
        {{
            instance = this;
        }}
        else if (instance != this)
        {{
            Destroy(gameObject);
        }}
        Time.timeScale = 0;
	}}

    public void OnStartGame()
    {{
        Time.timeScale = 1;
    }}

    public void OnGameOver()
    {{
        SceneManager.LoadScene("MainMap");
    }}

    public void BirdScored()
    {{
        if (gameOver)
        {{
            return;
        }}
        score++;
        scoreText.text = "Score: " + score.ToString();
    }}

    public void BirdDied()
    {{
        gameOver = true;       
        gameOverText.SetActive(true);
    }}
}}
```
- RepeatingBackground.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class RepeatingBackground : MonoBehaviour {{

    private BoxCollider2D groundCollider;
    private float groundHorizontalLength;
    
	// Use this for initialization
	void Start () 
    {{
        groundCollider = GetComponent<BoxCollider2D>();
        groundHorizontalLength = groundCollider.size.x;
	}}
	
	// Update is called once per frame
	void Update () 
    {{
		if (transform.position.x < -groundHorizontalLength)
        {{
            RepositionBackground ();
        }}
	}}

    private void RepositionBackground()
    {{
        Vector2 groundOffset = new Vector2(groundHorizontalLength * 2f, 0);
        transform.position = (Vector2)transform.position + groundOffset;
    }}
}}
```
- ScrollingObject.cs
```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class ScrollingObject : MonoBehaviour {{

    private Rigidbody2D rb2d;
	// Use this for initialization
	void Start () 
    {{
        rb2d = GetComponent<Rigidbody2D>();
        rb2d.velocity = new Vector2(GameControl.instance.scrollSpeed, 0);
	}}
	
	// Update is called once per frame
	void Update () 
    {{
		if (GameControl.instance.gameOver == true)
        {{
            rb2d.velocity = Vector2.zero;
        }}
	}}
}}
```

现在，请根据上述例子、用户的指示以及当前的游戏策划内容，先生成“所需代码”列表中第一个.cs代码文件的内容。
不要忘记，尽可能生成长而正确的代码，以保证策划案中的所有需求都能被正确实现。
"""

"当前用户希望实现的游戏策划和场景搭建如下："

'当前用户希望实现的游戏策划和场景搭建如下：'

In [ ]:
code_writing_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "code_writing_schema", 
        "schema": {
            "type": "object",
            "properties": {
                "filename": {
                    "type": "string",
                    "description": ".cs 文件名"
                },
                "code": {
                    "type": "string", 
                    "description": "全部 #c 代码"
                }
            },
            "required": ["filename", "code"]
        }
    }
}

In [15]:
from pydantic import BaseModel

class CodeWritingNode(BaseModel):


def generate_codebase_first():
    completion = client.beta.chat.completions.create(
        model="gpt-4o-2024-08-06",
        stream=False,
        messages=[{
            "role": "system",
            "content": code_base_prompt
        }],
        response_format=
    )

    # response_str = ""
    # for chunk in completion:
    #     if chunk.choices[0].delta.content is not None:
    #         content = chunk.choices[0].delta.content
    #         print(content, end="")
    #         response_str += content

    response_str = completion.choices[0].message.content
    print(response_str)

    # import json

    # def save_to_json(data, filename):
    #     with open(filename, 'w', encoding='utf-8') as f:
    #         json.dump(data, f, ensure_ascii=False, indent=4)

    # response_dict = json.loads(response_str)
    # save_to_json(response_dict, "codebase.json")

In [ ]:
generate_codebase_first()